# Equilibria Partitioning Example

Demonstrates using `LiquidVaporPartitioningStrategy` for computing
gas-particle equilibrium concentrations.

## What This Example Shows

- Creating a liquid-vapor partitioning strategy
- Solving for equilibrium with organic species
- Interpreting the equilibrium results
- Observing the effect of relative humidity on partitioning

In [1]:
import numpy as np
import particula as par

## 1. Define Organic Species Properties

Set up three organic species with different volatilities (C* values).
Lower C* means less volatile and more tendency to partition to the
particle phase.

In [2]:
# Three species with different volatilities (C* values)
c_star_j_dry = np.array([1e-6, 1e-4, 1e-2])  # saturation concentrations
concentration_organic = np.array([1.0, 5.0, 10.0])  # ug/m^3
molar_mass = np.array([200.0, 200.0, 200.0])  # g/mol
o2c_ratio = np.array([0.2, 0.3, 0.5])  # O:C ratios
density = np.array([1200.0, 1200.0, 1200.0])  # kg/m^3

print("=== Organic Species Properties ===")
print(f"C* (dry): {c_star_j_dry}")
print(f"Total concentration: {concentration_organic} ug/m^3")
print(f"O:C ratios: {o2c_ratio}")

=== Organic Species Properties ===
C* (dry): [1.e-06 1.e-04 1.e-02]
Total concentration: [ 1.  5. 10.] ug/m^3
O:C ratios: [0.2 0.3 0.5]


## 2. Create Partitioning Strategy at 75% RH

Water activity corresponds to relative humidity for aerosol systems.

In [3]:
strategy = par.equilibria.LiquidVaporPartitioningStrategy(
    water_activity=0.75,  # 75% relative humidity
)

print(f"Strategy: {type(strategy).__name__}")
print(f"Water activity (RH): {strategy.water_activity * 100:.0f}%")

Strategy: LiquidVaporPartitioningStrategy
Water activity (RH): 75%


## 3. Solve for Equilibrium

In [4]:
result = strategy.solve(
    c_star_j_dry=c_star_j_dry,
    concentration_organic_matter=concentration_organic,
    molar_mass=molar_mass,
    oxygen2carbon=o2c_ratio,
    density=density,
)

print("Equilibrium solved successfully!")

Equilibrium solved successfully!


## 4. Print Equilibrium Results

The result contains partition coefficients (fraction in condensed phase),
phase concentrations, and water content.

In [5]:
print("=== Liquid-Vapor Partitioning ===")
print(f"Water activity (RH): {strategy.water_activity * 100:.0f}%\n")

print("Input organic species:")
print(f"  C* (dry): {c_star_j_dry}")
print(f"  Total concentration: {concentration_organic} ug/m^3")
print(f"  O:C ratios: {o2c_ratio}")

print("\n=== Equilibrium Results ===")
print(f"Partition coefficients: {result.partition_coefficients}")
print("  (fraction in condensed phase)")

print("\nAlpha phase (water-rich):")
print(f"  Species concentrations: {result.alpha_phase.species_concentrations}")
print(
    "  Water concentration: "
    f"{result.alpha_phase.water_concentration:.2f} ug/m^3"
)
print(
    "  Total concentration: "
    f"{result.alpha_phase.total_concentration:.2f} ug/m^3"
)

if result.beta_phase is not None:
    print("\nBeta phase (organic-rich):")
    print(
        f"  Species concentrations: {result.beta_phase.species_concentrations}"
    )
    print(
        "  Water concentration: "
        f"{result.beta_phase.water_concentration:.2f} ug/m^3"
    )

print(
    f"\nWater content: alpha={result.water_content[0]:.2f}, "
    f"beta={result.water_content[1]:.2f} ug/m^3"
)
print(f"Optimization error: {result.error:.2e}")

=== Liquid-Vapor Partitioning ===
Water activity (RH): 75%

Input organic species:
  C* (dry): [1.e-06 1.e-04 1.e-02]
  Total concentration: [ 1.  5. 10.] ug/m^3
  O:C ratios: [0.2 0.3 0.5]

=== Equilibrium Results ===
Partition coefficients: [0.99999998 0.99999838 0.9998916 ]
  (fraction in condensed phase)

Alpha phase (water-rich):
  Species concentrations: [ 1.10751471  5.94511189 13.45712289]
  Water concentration: 4.51 ug/m^3
  Total concentration: 20.51 ug/m^3

Beta phase (organic-rich):
  Species concentrations: [0. 0. 0.]
  Water concentration: 0.00 ug/m^3

Water content: alpha=4.51, beta=0.00 ug/m^3
Optimization error: 5.57e-14


## 5. Effect of RH on Partitioning

As relative humidity increases, more water is absorbed and partitioning
changes.

In [6]:
print("=== Effect of RH on Partitioning ===\n")
rh_values = [0.3, 0.5, 0.75, 0.9]
for rh in rh_values:
    strat = par.equilibria.LiquidVaporPartitioningStrategy(
        water_activity=rh,
    )
    res = strat.solve(
        c_star_j_dry=c_star_j_dry,
        concentration_organic_matter=concentration_organic,
        molar_mass=molar_mass,
        oxygen2carbon=o2c_ratio,
        density=density,
    )
    mean_partition = np.mean(res.partition_coefficients)
    print(
        f"RH={rh * 100:.0f}%: mean partition coeff = {mean_partition:.4f}, "
        f"water = {res.alpha_phase.water_concentration:.2f} ug/m^3"
    )

=== Effect of RH on Partitioning ===

RH=30%: mean partition coeff = 0.9999, water = 0.66 ug/m^3
RH=50%: mean partition coeff = 0.9999, water = 1.52 ug/m^3
RH=75%: mean partition coeff = 1.0000, water = 4.51 ug/m^3
RH=90%: mean partition coeff = 1.0000, water = 13.42 ug/m^3
